In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import pandas as pd
import matplotlib as plt
from src.data.time_series_data import TimeSeriesData


In [3]:
path = "/Users/cooper/Desktop/precipitation-generator/data/raw/GSTEIGmeteo.csv"
ts_data = TimeSeriesData.from_csv(path)

In [4]:
ts_data.data

,precipitation
date,
1986-01-01,2.5
1986-01-02,0.2
1986-01-03,4.3
1986-01-04,3.4
1986-01-05,3.6
...,...
2000-09-26,0.0
2000-09-27,0.0
2000-09-28,0.0


In [5]:
ts_data.get_time_resolution()

'D'

In [6]:
weelkly_data = ts_data.resample_data("W")
weelkly_data.data

,precipitation
date,
1986-01-05,2.800000
1986-01-12,6.600000
1986-01-19,13.971429
1986-01-26,8.228571
1986-02-02,6.000000
...,...
2000-09-03,3.071429
2000-09-10,2.300000
2000-09-17,0.471429


In [7]:
ts_data.data

,precipitation
date,
1986-01-01,2.5
1986-01-02,0.2
1986-01-03,4.3
1986-01-04,3.4
1986-01-05,3.6
...,...
2000-09-26,0.0
2000-09-27,0.0
2000-09-28,0.0


In [9]:
filtered_data = ts_data.filter_by_date_range('1986-01-05', '1986-01-13')
filtered_data.data


,precipitation
date,
1986-01-05,3.6
1986-01-06,2.0
1986-01-07,0.5
1986-01-08,6.7
1986-01-09,2.6
1986-01-10,9.1
1986-01-11,16.0
1986-01-12,9.3
1986-01-13,31.5


In [11]:
precip_series = ts_data.get_precipitation_series()
precip_series

date
1986-01-01     2.5
1986-01-02     0.2
1986-01-03     4.3
1986-01-04     3.4
1986-01-05     3.6
              ... 
2000-09-26     0.0
2000-09-27     0.0
2000-09-28     0.0
2000-09-29     5.0
2000-09-30    30.4
Name: precipitation, Length: 5387, dtype: float64